In [60]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import json

In [61]:
NER = ["EFFECT"]
set = "test"

In [62]:
l = []
for ner in NER:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [63]:
df = df.reset_index().drop(columns=["index"])

In [64]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Taking, into, consideration, the, above, evid...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-1.8748829364776611, -1.967904806137085, 4.0...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
1,"[The, two, draft, genomes, (, Nitrospira, sp.,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.734492301940918, -2.349016666412353, 5.28...","[-100, 2, 2, 2, 2, 2, 2, -100, -100, 2, -100, ...","[None, 0, 1, 2, 3, 4, 5, 5, 5, 6, 6, 7, 8, 8, ...",EFFECT
2,"[Sle1, is, a, substrate, of, the, ClpXP, prote...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-1.911755681037902, -1.535628080368042, 3.48...","[-100, 2, -100, 2, 2, 2, 2, 2, 2, -100, -100, ...","[None, 0, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, ...",EFFECT
3,"[In, this, study, ,, we, sequenced, and, assem...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.416978597640991, -2.321799755096435, 5.10...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, -10...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 10,...",EFFECT
4,"[MF59, is, an, oil-in-water, emulsion, with, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-2.241502523422241, -2.455163955688476, 4.54...","[-100, 2, -100, 2, 2, 2, -100, -100, -100, -10...","[None, 0, 0, 1, 2, 3, 3, 3, 3, 3, 4, 5, 6, 7, ...",EFFECT
...,...,...,...,...,...,...,...
411,"[For, the, housekeeping, gene, GAPDH, ,, 20, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",411,"[[-2.416497945785522, -1.909170269966125, 4.49...","[-100, 2, 2, 2, -100, -100, 2, 2, 2, 2, 2, -10...","[None, 0, 1, 2, 2, 2, 3, 4, 5, 6, 7, 7, 7, 8, ...",EFFECT
412,"[Chemotaxonomic, analyses, showed, that, type,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",412,"[[-2.4317431449890132, -2.126706838607788, 4.5...","[-100, 2, -100, -100, 2, 2, 2, 2, 2, 2, -100, ...","[None, 0, 0, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, ...",EFFECT
413,"[[, 6, ], Some, volatile, sesquiterpenoids, ar...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",413,"[[-2.476452589035034, -1.915351033210754, 3.87...","[-100, 2, 2, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 10...",EFFECT
414,"[Nanosynbacter, lyticus, stain, TM7x, of, the,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",414,"[[-2.470869064331054, -2.453198671340942, 5.44...","[-100, 2, -100, -100, -100, -100, 2, -100, 2, ...","[None, 0, 0, 0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 5, ...",EFFECT


In [65]:
label_ids = df[["label_ids"]]

In [66]:
annotations = []
for j in range(len(df)):
	for i in zip(df.loc[j,"tokens"],df.loc[j,"ner_tags"]):
		if i[1] != "O":
			annotations.append({"label":i[1],"text":i[0]})

### Predict sentences again

In [96]:
ner = "EFFECT"

In [97]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=1)


In [98]:
import jsonlines

# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

predictions = []
# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:

            # Get the value from the "text" field
            text = l["data"]["text"]
            # Pass the value to the nlp() function
            print(text)
            predict = nlp(text)
            predictions.append(predict)
            print(predict)

Moreover, due to enormous metabolic flexibility, S. thermodepolymerans DSM 15344 is capable of PHA production from human food chain noncompeting substrates such as lignocelluloses and other waste streams of the agro-food industry.
[]
Therefore, this study aimed to reveal the potential of Citrobacter werkmanii strain WWN1 and Enterobacter cloacae strain JWM6 isolated from agricultural soils irrigated with wastewater to mitigate multi-metal toxicity in wheat.
[{'entity_group': 'B', 'score': 0.4811349, 'word': 'metal', 'start': 203, 'end': 208}]
Furthermore, robust induction of mRFP1 expression while encased in the pellicle indicates that, despite entrapped within nanocellulose, the K. intermedius ENS15 cells can effectively receive signals from their environment enabling its applicability in applications requiring long-term cell survival and tolerance to toxic compounds (Fig 5 d,e).
[]
Together, these results confirm that the 4KO+EaDAcT strain produces enriched amounts of acetyl-TAGs, bu

In [99]:
predictions = [p for p in predictions if p]

In [100]:
predictions

[[{'entity_group': 'B',
   'score': 0.4811349,
   'word': 'metal',
   'start': 203,
   'end': 208}],
 [{'entity_group': 'B',
   'score': 0.60063523,
   'word': 'doripenem',
   'start': 210,
   'end': 219},
  {'entity_group': 'B',
   'score': 0.7206792,
   'word': 'doxycycline',
   'start': 221,
   'end': 232},
  {'entity_group': 'B',
   'score': 0.4791887,
   'word': 'ertapenem',
   'start': 234,
   'end': 243},
  {'entity_group': 'B',
   'score': 0.69496006,
   'word': 'erythromycin',
   'start': 245,
   'end': 257},
  {'entity_group': 'B',
   'score': 0.7547156,
   'word': 'gentamicin',
   'start': 259,
   'end': 269},
  {'entity_group': 'B',
   'score': 0.7179205,
   'word': 'imipenem',
   'start': 271,
   'end': 279},
  {'entity_group': 'B',
   'score': 0.69344217,
   'word': 'kanamycin',
   'start': 281,
   'end': 290},
  {'entity_group': 'B',
   'score': 0.66386074,
   'word': 'meropenem',
   'start': 292,
   'end': 301},
  {'entity_group': 'B',
   'score': 0.62491924,
   'word':

In [101]:
annotations

[{'label': 'B', 'text': 'activity'},
 {'label': 'B', 'text': 'β-lactam'},
 {'label': 'I', 'text': 'hyper-resistance'},
 {'label': 'B', 'text': 'resistance'},
 {'label': 'I', 'text': 'to'},
 {'label': 'I', 'text': 'extreme'},
 {'label': 'I', 'text': 'iron'},
 {'label': 'I', 'text': 'concentrations'},
 {'label': 'B', 'text': 'ISR'},
 {'label': 'B', 'text': 'neuroprotective'},
 {'label': 'I', 'text': 'effects'},
 {'label': 'B', 'text': 'neuronal'},
 {'label': 'I', 'text': 'health'},
 {'label': 'B', 'text': 'lateral'},
 {'label': 'I', 'text': 'root'},
 {'label': 'I', 'text': 'formation'},
 {'label': 'B', 'text': 'auxin'},
 {'label': 'I', 'text': 'biosynthesis'},
 {'label': 'B', 'text': 'confers'},
 {'label': 'I', 'text': 'resistance'},
 {'label': 'I', 'text': 'to'},
 {'label': 'I', 'text': 'Pb'},
 {'label': 'I', 'text': '('},
 {'label': 'I', 'text': 'II'},
 {'label': 'I', 'text': ')'},
 {'label': 'I', 'text': 'salts'},
 {'label': 'B', 'text': 'non-adherent'},
 {'label': 'I', 'text': 'growi

In [105]:
for prediction in predictions:
    pred = prediction[0]["word"]
    count = 0
    for annotation in annotations:
        ann = annotation["text"]
        if ann in pred:
            count +=1
    if count <1:
        print(pred)

doripenem
el42
kcl
copper
sodium
zn
copper
n
co
o
co
sodium
ii
ni
copper
mn
5


In [114]:
ners =["COMPOUND", "DISEASE", "EFFECT","ISOLATE","MEDIUM","METABOLITE","ORGANISM",
            "PHENOTYPE","PROTEIN","SPECIES","STRAIN"]
for ner in ners:
    test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
    test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
    merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
    merged.loc[:,"ner"] = ner
    label_ids = merged[["label_ids"]]

    annotations = []
    for j in range(len(merged)):
    	for i in zip(merged.loc[j,"tokens"],merged.loc[j,"ner_tags"]):
    		if i[1] != "O":
    			annotations.append({"label":i[1],"text":i[0]})
    
    path= f"../NER_output/{ner}"
    
    tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
    model = AutoModelForTokenClassification.from_pretrained(path)
    
    nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
                   aggregation_strategy="average", device=1)

    # Specify the path to the JSONL file
    jsonl_file = f"../NER/{ner}/test.jsonls"
    
    predictions = []
    # Open the JSONL file
    with jsonlines.open(jsonl_file) as reader:
        # Iterate over each line in the file
        for line in reader:
            for l in line:
    
                # Get the value from the "text" field
                text = l["data"]["text"]
                # Pass the value to the nlp() function
                predict = nlp(text)
                predictions.append(predict)
    predictions = [p for p in predictions if p]
    print("-----")
    print(ner)
    print("-----")
    for prediction in predictions:
        pred = prediction[0]["word"]
        count = 0
        for annotation in annotations:
            ann = annotation["text"].lower()
            if ann in pred:
                count +=1
        if count ==0:
            print(pred)

/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
COMPOUND
-----
doripenem
h
el42
h
ant44
n
co
h
nemafric
o
co
5


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
DISEASE
-----
uc
lung
axial


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

-----
PROTEIN
-----
mrp
β
α
e
np
α


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
SPECIES
-----
a
p
p
m
l
m
r
a
e
a
n
m
p
s
p
s
r
l
k
v
e
l
s
a
s
x
p
metallidurans
x
e
m
a
flexner
m
v


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
STRAIN
-----
s
l
p
b
l
v
osish
l
rh
vu
p
osish
b
b
gt
m
n
t
m
p
s
p
y
b
v
b
k
shli
l
l
b
b
l
p
db
b
s
v
osish
v
p
b
b
p
g
b
v
d
osish
p
w
b
v
br
osish
osish
l
p
p
